# Trie <small>(a little tenderness)</small>

Vous aviez à faire l'exercice suivant : « Vous écrirez un script qui reçoit en argument un fichier csv et un préfixe. Le script renverra tous les mots du fichier qui commencent par ce préfixe. Vous pourrez travailler sur `austronesian_swadesh.csv` ou `Lexique383.tsv` par exemple. »

Voilà ce qu'on aurait pu faire. En deux temps :
  1. Stockage des mots de Lexique.org

In [105]:
import csv

lexique = '/home/clement/l-pro/rsrc/lexique.org/Lexique383.tsv'
words_lexique = []
with open(lexique) as data:
    reader = csv.DictReader(data, delimiter="\t")
    for item in reader:
        words_lexique.append(item['ortho'])
print(f"On a {len(words_lexique)} mots en magasin.")

On a 142694 mots en magasin.


  2. Fonction de recherche par préfixe
  
  C'est la fonction de recherche qui va surtout nous intéresser aujourd'hui.

In [104]:
def search_prefix(words, prefix):
    """
    Search all the string beginning with the given prefix in
    a list of words
    Args:
        words: list of words
        prefix: string to search
    Returns:
        list of words
    Raises:
        KeyError: if prefix not found
    """
    res = []
    for word in words:
        if len(prefix) > len(word):
            continue
        elif word[:len(prefix)] == prefix:
            res.append(word)
    if res:
        return res
    else:
        raise KeyError(f"No word with prefix {prefix}")

In [107]:
print(search_prefix(words_lexique, "const"))

['constable', 'constamment', 'constance', 'constances', 'constant', 'constante', 'constante', 'constantes', 'constantes', 'constants', 'constat', 'constat', 'constata', 'constatai', 'constataient', 'constatais', 'constatait', 'constatant', 'constatation', 'constatations', 'constate', 'constatent', 'constater', 'constatera', 'constaterai', 'constateraient', 'constaterais', 'constaterait', 'constateras', 'constaterez', 'constateriez', 'constatez', 'constatiez', 'constations', 'constatons', 'constats', 'constats', 'constatâmes', 'constatèrent', 'constaté', 'constatée', 'constatées', 'constatés', 'constellaient', 'constellait', 'constellant', 'constellation', 'constellations', 'constelle', 'constellent', 'constellé', 'constellé', 'constellée', 'constellée', 'constellées', 'constellées', 'constellés', 'consterna', 'consternait', 'consternant', 'consternant', 'consternante', 'consternantes', 'consternants', 'consternation', 'consterne', 'consternent', 'consterné', 'consterné', 'consternée', 

In [108]:
%timeit search_prefix(words_lexique, "const")

39.2 ms ± 998 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Voyons ce que ça donne si on prend un lexique plus grand, [lefff](http://pauillac.inria.fr/~sagot/#lefff) par exemple.

In [109]:
lexique = '/home/clement/l-pro/rsrc/lefff-3.4.elex/lefff-3.4.elex'
words_lefff = []
with open(lexique, encoding="iso-8859-1") as data:
    reader = csv.reader(data, delimiter="\t")
    for item in reader:
        words_lefff.append(item[0])
print(f"On a {len(words_lefff)} mots en magasin.")

On a 1197641 mots en magasin.


In [110]:
%timeit search_prefix(words_lefff, "conv")

308 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


27.9 ms pour 142 694 mots, 203 ms pour 1 197 641 mots. Ce qui paraît assez logique, la fonction fait autant de tours de boucles qu'il y a de mots dans la liste. Le temps de traitement est directement proportionnel à la taille des données.

Cela nous permet de déterminer [la complexité en temps](https://fr.wikipedia.org/wiki/Complexit%C3%A9_en_temps) de notre algorithme. C'est une notion particulièrement importante en informatique, on note la complexité avec le « grand O » $O$. On parle de temps mais c'est le nombre d'étapes de calcul plus que le temps de calcul que l'on détermine avec la complexité.  
Ici notre algo a une complexité linéaire $O$ (où n est la taille des données).

La recherche de mots à partir de préfixe, nous l'utilisons quotidiennement avec la complétion automatique sur les claviers de téléphone ou les suggestions de requête sur les moteurs de recherche. Deux opérations a priori anodines mais où le temps de traitement est crucial pour l'expérience utilisateur. Un algo avec une complexité $O(n)$ n'est clairement pas adapté.

Il existe une structure de données quasi dédiée à ce type d'opérations : le [Trie](https://fr.wikipedia.org/wiki/Trie_(informatique)) ou arbre préfixe ou encore arbre lexicographique.

## Trie <small>(again)</small>

Un *trie* est un arbre de la famille des arbres de recherche (*search tree*). L'arbre stocke un ensemble de mots, un caractère par arc. Tous les descendants d'un nœud ont le même préfixe. Chaque mot du dictionnaire correspond à un chemin de la racine vers un nœud de l'arbre.

In [42]:
from IPython.display import Image
Image(url='https://brilliant-staff-media.s3-us-west-2.amazonaws.com/tiffany-wang/tb4AvuIQIg.png', width=400, height=800)

Il existe aussi une visualisation très éclairante ici : [https://www.cs.usfca.edu/~galles/visualization/Trie.html](https://www.cs.usfca.edu/~galles/visualization/Trie.html)  
Chaque caractère est stocké dans un nœud. C'est à peu près comme ça que nous allons implémenter le *trie*.

### Implémentation en Python

Vous connaissez déjà les structures de données de Python. L'idée maintenant est d'implémenter d'autres structures de données à l'aide des « briques » de base : `list`, `set` et `dict` de Python.

Nous allons faire une implémentation en objet en définissant ce qu'est un nœud puis un *trie*

In [141]:
class Node:
    """
    Un nœud de trie. Il a un caractère et des nœuds enfants (vide à l'initialisation)
    """
    def __init__(self, char):
        self.value = char
        self.children = {}
        self.end = False

In [149]:
class Trie:
    """
    trie
    """
    def __init__(self):
        """
        À l'initialisation on a juste un nœud racine vide
        """
        self.root = Node("")
        
    def insert(self, word):
        """
        Un nouveau mot dans le trie
        """
        node = self.root
        
        # on parcourt car. par car.
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                # Si un caractère du mot est inconnu
                # on ajoute un nouveau nœud
                new_node = Node(char)
                node.children[char] = new_node
                node = new_node
        node.end = True
        
    def _dfs(self, node, prefix):
        """
        parcours du trie en profondeur
        
        Args:
            - node: le nœud de départ
            - prefix: le préfixe, point de départ de la recherche
        """
        # cas du nœud terminal
        # if not(node.children):
        if node.end:
            self.output.append(prefix + node.value)
        
        for child in node.children.values():
            self._dfs(child, prefix + node.value)
            
    def search(self, prefix):
        """
        Cherche tous les mots du trie formés avec le préfixe argument
        """
        self.output = []
        node = self.root
        
        # on se déplace jusqu'au nœud de fin de préfixe
        for char in prefix:
            if char in node.children:
                node = node.children[char]
            else:
                # pas de préfixe, pas de résultat
                return []
        
        # appel à la fonction récursive
        self._dfs(node, prefix[:-1])
        return self.output


Comparons sur les mots de lexique.org et du lefff

In [143]:
t_lexique = Trie()
for w in words_lexique:
    t_lexique.insert(w)

In [151]:
%timeit t_lexique.search('mais')

10.4 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [113]:
t_lefff = Trie()
for w in words_lefff:
    t_lefff.insert(w)

In [116]:
%timeit t_lefff.search('conv')

608 µs ± 145 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


C'est beaucoup plus rapide que la méthode basée sur une liste évidemment et surtout la taille du lexique influe peu sur le temps de traitement. Un *trie* a une complexité $O(m)$ où m est la taille du préfixe recherché.  
Essayez avec un préfixe court vous verrez que le temps de traitement est plus long.

### Et si je veux conserver en mémoire le *trie* ?

Pour les structures de données de base vous pouvez utiliser JSON, format que vous connaissez. Pour les objets ça n'ira pas.

In [128]:
import json

json.dump(words_lexique, open('words_lexique.json', 'w'))

In [132]:
json.dump(t_lexique, open('t_lexique.json', 'w'))

TypeError: Object of type Trie is not JSON serializable

Pour sérialiser et désérialiser les objets vous pouvez utiliser le module `pickle`  
Attention de bien utiliser le mode binaire pour l'ouverture du fichier d'export.

In [137]:
import pickle

pickle.dump(t_lexique, open('t_lexique.pickle', 'wb'))

In [148]:
t_lexique_2 = pickle.load(open('t_lexique.pickle', 'rb'))
type(t_lexique_2)

__main__.Trie